# **Instruções Gerais**

- **Copie** este notebook, junto com as imagens da pasta para o seu Google Drive da UFV
- Resolva o que está sendo pedido no próprio notebook
- Lembre-se de **montar o seu Google Drive** no sistema de arquivos do notebook (ícone da pastinha, na barra de ferramentas à esquerda)
- Entregue, como resposta à atividade no **Moodle**

- **Obs.:**
  - Lembre-se de **salvar** o notebook de tempos em tempos
  - Envie **apenas** o notebook. Não há necessidade de enviar as imagens

# **1. Preâmbulo**

## **1.1. Identifique-se**

Complete o que está sendo pedido na célula a seguir

In [ ]:
# Informe, dentro das strings, seu nome e número de matrícula
estudante = 'Erick Lima Figueiredo'
matricula = '98898'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## **1.2. Importações de módulos Python**

- As bibliotecas necessárias para a atividade já se encontram importadas na célula a seguir. Mas você pode adicionar novas importações, se achar necessário

In [ ]:
from PIL import Image
import numpy as np
from scipy.interpolate import RectBivariateSpline

## **1.3. Funções auxiliares já implementadas**

- Para informações mais detalhadas a respeito das funções, consulte o notebook de aula do capítulo "Entropia, Ruídos e Métricas"

### **1.3.1. Função `gera_imagem`**

In [ ]:
def gera_imagem(arr_orig):
  arr = arr_orig.copy()
  arr = np.round(arr)
  arr[arr > 255] = 255
  arr[arr < 0] = 0
  return Image.fromarray(arr.astype(np.uint8))

## **1.4. Inicialização de *path***

- Lembre-se de alterar o valor da string de path para a pasta do seu Drive onde você salvou o notebook e as imagens.
- O caminho da string deve terminar com um caractere de barra (`/`), para que o restante do código funcione sem alterações.

In [ ]:
path = '/content/drive/MyDrive/Praticas PDI/'

# **2. Exercícios**

## **2.1. Cisalhamento vertical**

Com base no que foi dado em sala de aula, implemente o cisalhamento vertical com e sem extensão das dimensões da imagem resultante.

Neste notebook, como vamos lidar com o redimensionamento de imagens, não vamos utilizar a função `exibe_mosaico`, que nivela os tamanhos das imagens resultantes. Portanto, vamos utilizar apenas a exibição dos resultados usando a função `display` nativa dos notebooks.

#### **Preparando os testes**

Execute a célula abaixo para visualizar a imagem original.


In [ ]:
img = Image.open(path + 'capivara_salto.png').convert('RGB')
display(img)

#### **Exercício:**

Implemente o corpo da função `cisalha_vert` abaixo. A função terá como parâmetros:

- `img`: um objeto `PIL Image`
- `beta`: fator de cisalhamento ($\in[0, 1]$)
- `expande`: *flag* indicando se a imagem resultante deve ser expandida para conter todos pixels da matriz original (`True`) ou não (`False`)
- `inverte`: *flag* indicando se o cisalhamento acontecerá de baixo para cima (`True`) ou ao contrário (`False`).

Os dois últimos parâmetros (`expande` e `inverte`) são opcionais e têm valores *default* falsos. A função deve retornar uma imagem `PIL`.

In [ ]:
def cisalha_vert(img, beta, expande=False, inverte=False):
  #vertical = p(x,y+f*x)
  img_arr = np.asarray(img)

  nova_altura = round(img_arr.shape[0] + beta * img_arr.shape[1])

  arr_cisal = np.zeros((nova_altura if expande else img_arr.shape[0], img_arr.shape[1], img_arr.shape[2]))

  x_limit = img_arr.shape[1]-1

  for i in range(img_arr.shape[0]):
    for j in range(img_arr.shape[1]):
      I = int((i + beta*(x_limit-j)) if inverte else (i + beta*j))

      if expande or 0 <= I <= img_arr.shape[0]-1:
        arr_cisal[I, j] = img_arr[i,j]
      
  return gera_imagem(arr_cisal)
  

#### **Testando sua solução**

Execute a célula a seguir.

In [ ]:
display(img)
print('Sem inversão nem expansão da imagem:')
imgCisalha = cisalha_vert(img, beta=0.5, expande=False, inverte=False)
display(imgCisalha)
print('Com inversão e sem expansão da imagem')
imgCisalha = cisalha_vert(img, beta=0.5, expande=False, inverte=True)
display(imgCisalha)
print('Sem inversão e com expansão da imagem')
imgCisalha = cisalha_vert(img, beta=0.5, expande=True, inverte=False)
display(imgCisalha)
print('Com inversão e expansão da imagem')
imgCisalha = cisalha_vert(img, beta=0.5, expande=True, inverte=True)
display(imgCisalha)
print('Sem inversão nem expansão da imagem e beta diferente')
imgCisalha = cisalha_vert(img, beta=0.3, expande=False, inverte=False)
display(imgCisalha)

#### **Observação**

Você deve ter notado um problema com o resultado do cisalhamento aplicado nesta imagem, certo?

É o mesmo fenômeno que acontece na rotação da imagem, conforme vimos em sala. E a solução é a mesma: interpolação.

Uma possível solução para corrigir este problema está disponibilizada em material à parte, a título de material complementar e não será cobrado nesta atividade. Deixemos os "buracos" gerados pelo cisalhamento como estão.

## **2.2. Redimensionamento**

Implemente o corpo da função `redim` abaixo, que faz, por meio de **interpolação**, o redimensionamento de imagens. Os parâmetros são:

- `img`: a imagem de entrada (`PIL Image`)
- `alfa`: fator de redimensionamento, onde $\alpha > 0$. $\alpha$ trata-se, portanto, de um valor **percentual** de redimensionamento

Note que:

- Para $0 < \alpha < 1$, aplica-se uma *redução* no tamanho da imagem
- Para $\alpha > 1$, aplica-se uma *ampliação* no tamanho da imagem
- Para $\alpha = 1$, nenhum efeito é aplicado
- O aspecto, isto é, as proporções da imagem original devem ser mantidas. Em outras palavras, aplica-se o mesmo fator de redimensionamento $\alpha$ em ambas dimensões (altura e largura) da imagem.
- Após utilizar $\alpha$ para calcular as dimensões da nova imagem, **arredonde** os valores obtidos (não trunque).

Retorno: imagem `PIL Image` redimensionada.

#### **Algoritmo em alto nível**

Por se tratar de um exercício menos trivial, um algoritmo em alto nível para a solução é dado desta vez:

1. Calcule as novas altura e largura da imagem resultante

2. Crie vetores x e y com os índices que correspondem às linhas e às colunas da imagem original

3. Crie novos vetores x' e y' utilizando os valores limites de x e y, e com as novas quantidades de valores intermediários entre estes limites

4. Crie uma imagem nova, com as novas dimensões

5. Para cada canal da imagem original:

  a. Obtenha um interpolador a partir de x, y e os dados do canal

  b. Produza novos dados (canal'), via interpolação, a partir de x' e y'

  c. Atribua canal' ao canal correspondente da imagem resultante

6. Produza e retorne uma imagem a partir da matriz interpolada

In [ ]:
def redim(img, alfa):
  img_arr = np.asarray(img)

  x = np.arange(img_arr.shape[0])
  y = np.arange(img_arr.shape[1])

  nv_x = np.linspace(np.min(x), np.max(x), round(alfa * img_arr.shape[0]-1))
  nv_y = np.linspace(np.min(y), np.max(y), round(alfa * img_arr.shape[1]-1))

  x_interpolator = RectBivariateSpline(x, y, img_arr[:,:,0])
  y_interpolator = RectBivariateSpline(x, y, img_arr[:,:,1])
  z_interpolator = RectBivariateSpline(x, y, img_arr[:,:,2])

  res = np.dstack((x_interpolator(nv_x, nv_y), 
                  y_interpolator(nv_x, nv_y),
                  z_interpolator(nv_x, nv_y)))
  
  return gera_imagem(res)


#### **Testando sua solução**

Execute a célula a seguir.

In [ ]:
img = Image.open(path + 'pequenina.png')
imgRedim = redim(img, 3)

print('Imagem Original')
display(img)
print('Imagem Nova')
display(imgRedim)

img = Image.open(path + 'por_do_sol.png')
imgRedim = redim(img, 0.5)

print('Imagem Original')
display(img)
print('Imagem Nova')
display(imgRedim)

#### **Bons estudos!**

- Lembre-se de enviar **apenas** o notebook
- **Limpe as saídas em tela** antes de enviar. Isso diminui significativamente o tamanho do arquivo final e não gera problemas de recusa de submissão no Moodle. Para isso:
  - Vá no menu *Editar*
  - Escolha a opção *Limpar todas as saídas*
  - Salve o notebook